In [1]:
import pandas as pd
import glob
import shutil
import sys
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import math
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_pickle('df.pkl')

In [ ]:
df.to_pickle('df.pkl')

In [ ]:
# Categorical features 
cat_features = features.select_dtypes(include=['object']).columns
print(cat_features)

for y in cat_features:
    features = features.join(pd.get_dummies(features[y], prefix=y))
    features.drop(y, axis=1, inplace=True)

In [ ]:
# Split the data in features and target label
label = df['HasDetections']
features = df.drop('HasDetections', axis=1)

In [10]:
# Percentage of nulls
print(type(df.isnull().sum()/df.shape[0]))
plt.figure()
# (df.isnull().sum()/df.shape[0]).plot.hist()
print((df.isnull().sum()/df.shape[0]).sort_values(ascending=False))

<class 'pandas.core.series.Series'>
Census_OSVersion_3                                   0.0
Census_PrimaryDiskTypeName                           0.0
UacLuaenable                                         0.0
Census_MDC2FormFactor                                0.0
Census_DeviceFamily                                  0.0
Census_OEMNameIdentifier                             0.0
Census_OEMModelIdentifier                            0.0
Census_ProcessorCoreCount                            0.0
Census_ProcessorManufacturerIdentifier               0.0
Census_ProcessorModelIdentifier                      0.0
Census_PrimaryDiskTotalCapacity                      0.0
Census_SystemVolumeTotalCapacity                     0.0
SmartScreen                                          0.0
Census_HasOpticalDiskDrive                           0.0
Census_TotalPhysicalRAM                              0.0
Census_ChassisTypeName                               0.0
Census_InternalPrimaryDiagonalDisplaySizeInInches   

<Figure size 432x288 with 0 Axes>

In [ ]:
# Remove the rows, where the value count is less than 1000
print(df.Census_InternalBatteryType.value_counts())
# vc = df.Census_InternalBatteryType.value_counts()
# u  = [i not in set(vc[vc<1000].index) for i in df['Census_InternalBatteryType']]
# df = df[u]
print(df.Census_InternalBatteryType.value_counts())

In [ ]:
df.drop(df[df.SmartScreen == '&#x01;'].index, inplace=True)
df.drop(df[df.SmartScreen == '&#x02;'].index, inplace=True)
df.drop(df[df.SmartScreen == '&#x03;'].index, inplace=True)
df.drop(df[df.SmartScreen == '0'].index, inplace=True)
df.drop(df[df.SmartScreen == '00000000'].index, inplace=True)
df.drop(df[df.SmartScreen == 'Enabled'].index, inplace=True)
print(df['SmartScreen'].unique())
df.SmartScreen.value_counts()

In [ ]:
print(df['SmartScreen'].unique())
df.SmartScreen.value_counts()

In [ ]:
# Percentage of nulls
print(type(df.isnull().sum()/df.shape[0]))
plt.figure()
# (df.isnull().sum()/df.shape[0]).plot.hist()
print((df.isnull().sum()/df.shape[0]).sort_values(ascending=False))
(df.isnull().sum()/df.shape[0]).sort_values().plot()
plt.show()

In [ ]:
for f in drop_list:
    print('Dropping: ' + col)
    df = df.drop(f, 1)

In [ ]:
# Remove columns with only 1 unique value
# Should do in the end also.
# TODO
# df.dropna(thresh=2)
for col in df:
    if(len(df.loc[:,col].unique()) == 1):
        print('Dropping: ' + col)
        drop_list.append(col)

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
# start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
# print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
df['Census_OSVersion_0'] = df['Census_OSVersion'].apply(break_and_get_part, position=0)
df['Census_OSVersion_1'] = df['Census_OSVersion'].apply(break_and_get_part, position=1)
df['Census_OSVersion_2'] = df['Census_OSVersion'].apply(break_and_get_part, position=2)
df['Census_OSVersion_3'] = df['Census_OSVersion'].apply(break_and_get_part, position=3)

drop_list.append('Census_OSVersion')

print(df['Census_OSVersion_0'].unique())
print(df['Census_OSVersion_1'].unique())
print(df['Census_OSVersion_2'].unique())
print(df['Census_OSVersion_3'].unique())

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [ ]:
df.loc[df.SmartScreen == 'off', 'SmartScreen'] = "Off"
df.loc[df.SmartScreen == 'on', 'SmartScreen'] = "On"
df.loc[df.SmartScreen == 'OFF', 'SmartScreen'] = "Off"
df.loc[df.SmartScreen == 'requireAdmin', 'SmartScreen'] = "RequireAdmin"
df.loc[df.SmartScreen == 'Promt', 'SmartScreen'] = "Prompt"
df.loc[df.SmartScreen == 'prompt', 'SmartScreen'] = "Prompt"
df.loc[df.SmartScreen == 'warn', 'SmartScreen'] = "Warn"
df.loc[df.SmartScreen == 'requireadmin', 'SmartScreen'] = "RequireAdmin"
print(df['SmartScreen'].unique())

In [ ]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

In [9]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

   UacLuaenable Census_MDC2FormFactor Census_DeviceFamily  \
0           1.0               Desktop     Windows.Desktop   
1           1.0              Notebook     Windows.Desktop   
2           1.0               Desktop     Windows.Desktop   
3           1.0               Desktop     Windows.Desktop   
4           1.0              Notebook     Windows.Desktop   

   Census_OEMNameIdentifier  Census_OEMModelIdentifier  
0                    2668.0                     9124.0  
1                    2668.0                    91656.0  
2                    4909.0                   317701.0  
3                    1443.0                   275890.0  
4                    1443.0                   331929.0  
UacLuaenable float64
[1.0000000e+00 0.0000000e+00 4.8000000e+01 3.0000000e+00 2.0000000e+00
 6.3570620e+06 4.9000000e+01 1.6777216e+07 5.0000000e+00 2.5500000e+02
 7.7988840e+06]
Census_MDC2FormFactor object
['Desktop' 'Notebook' 'Detachable' 'PCOther' 'AllInOne' 'Convertible'
 'SmallTablet

In [ ]:
# print(df['OsBuildLab'].dtype)
# print(df['OsBuildLab'].unique())
df['OsBuildLab_0'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=0)
df['OsBuildLab_1'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=1)
df['OsBuildLab_2'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=2)
df['OsBuildLab_3'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=3)
df['OsBuildLab_4'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=4)
df['OsBuildLab_5'] = df['OsBuildLab'].apply(break_and_get_part_osbuildlab, position=5)
drop_list.append('OsBuildLab')
print(df['OsBuildLab_0'].unique())
print(df['OsBuildLab_1'].unique())
print(df['OsBuildLab_2'].unique())
print(df['OsBuildLab_3'].unique())
print(df['OsBuildLab_4'].unique())
print(df['OsBuildLab_5'].unique())

In [ ]:
import math
def break_and_get_part_osbuildlab(value, position):
    if position < 4:
        try:
            return value.split(".")[position]
        except:
            return math.nan
    elif position == 4:
        try:
            return value.split(".")[4].split('-')[0]
        except:
            return math.nan
    elif position == 5:
        try:
            return value.split(".")[4].split('-')[1]
        except:
            return math.nan

In [ ]:
df['OsVer_0'] = df['OsVer'].apply(break_and_get_part, position=0)
df['OsVer_1'] = df['OsVer'].apply(break_and_get_part, position=1)
df['OsVer_2'] = df['OsVer'].apply(break_and_get_part, position=2)
df['OsVer_3'] = df['OsVer'].apply(break_and_get_part, position=3)
drop_list.append('OsVer')
print(df['OsVer_0'].unique())
print(df['OsVer_1'].unique())
print(df['OsVer_2'].unique())
print(df['OsVer_3'].unique())

In [8]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

   AutoSampleOptIn  SMode  IeVerIdentifier   SmartScreen  Firewall
0                0    0.0            137.0             0       1.0
1                0    0.0            137.0             0       1.0
2                0    0.0            137.0  RequireAdmin       1.0
3                0    0.0            137.0  ExistsNotSet       1.0
4                0    0.0            137.0  RequireAdmin       1.0
AutoSampleOptIn int64
[0 1]
SMode float64
[0. 1.]
IeVerIdentifier float64
[137.  94.   0. 290. 117. 111. 333.  65. 108.  82. 103.  98. 323.  78.
 105.  84. 135.  41.  76.  53. 107.  74. 302.  96.  68. 114. 311.  51.
  71.  81. 335.  42.  46. 308. 334.  44.  90. 325.  88. 295.  92. 185.
  87.  85. 327.  73.  86. 163.  91. 309. 337. 336. 317. 180. 190. 324.
  50.  72. 321.  48.  64. 196. 331.  63.  45.  49. 318. 312. 326. 174.
 176. 305. 287. 315. 186. 313. 194.  77. 169. 181. 332. 288. 153. 199.
  58.  39. 205. 328. 300. 347. 307. 102. 152. 275. 303. 319. 322. 384.
 294. 178. 154. 182. 158. 1

In [7]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

   OsBuild  OsSuite OsPlatformSubRelease SkuEdition  IsProtected
0    17134      256                  rs4        Pro          1.0
1    17134      256                  rs4        Pro          1.0
2    17134      768                  rs4       Home          1.0
3    17134      256                  rs4        Pro          1.0
4    17134      768                  rs4       Home          1.0
OsBuild int64
[17134 14393 16299  7601  9600 10240 15063 10586 17744 17692 17738 17751
 17755 17730 17733 17746 17735 17754 17741 17713 17740 17672 17758 17763
  7600 17666 17677 17686 17747 17760 17759 17682 17761 17704 17661 17643
 18234 17618 18204 17639 17634 17728 18237 18242 17133 17723 17650 17711
 18219 18240 17627 17655 18214 18201 17604 18224 18241 17689 17718 17753
 18227 18238 17725 17694 17757 18233 17623 17756 17749 17734 17752 18226
 18230 18244 18236 17750]
OsSuite int64
[256 768 272  16 400 784 305 274  49 144 402 528 307  18]
OsPlatformSubRelease object
['rs4' 'rs1' 'rs3' 'windows7' 'w

In [6]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

   OrganizationIdentifier  GeoNameIdentifier  LocaleEnglishNameIdentifier  \
0                    18.0               35.0                          171   
1                    18.0              119.0                           64   
2                    18.0               64.0                           49   
3                     0.0              117.0                          115   
4                     0.0              277.0                           75   

    Platform Processor  
0  windows10       x64  
1  windows10       x64  
2  windows10       x64  
3  windows10       x64  
4  windows10       x64  
OrganizationIdentifier float64
[18.  0. 27. 46. 11. 14. 37. 10. 50. 49. 33.  8. 48. 36. 31.  4.  1. 28.
  3. 52. 32. 51.  5.  2. 47. 44. 16. 40. 20. 22. 29. 26. 21. 39.  6. 19.
  7. 30. 42. 43. 41. 15. 45. 25. 35. 23. 38. 12. 17. 34.]
GeoNameIdentifier float64
[ 35. 119.  64. 117. 277. 126.  89. 205. 138. 101. 211. 209. 199. 230.
 202.  53. 187.  15. 255. 274. 266.  10. 276. 267. 240.

In [5]:
# Now analyze the next 5 features
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
  print(y, df[y].dtype)
  print(df[y].unique())
start_cnt += show_cnt

   AVProductsInstalled  AVProductsEnabled  HasTpm  CountryIdentifier  \
0                  1.0                1.0       1                 29   
1                  1.0                1.0       1                 93   
2                  1.0                1.0       1                 86   
3                  1.0                1.0       1                 88   
4                  1.0                1.0       1                 18   

   CityIdentifier  
0        128035.0  
1          1482.0  
2        153579.0  
3         20710.0  
4         37376.0  
AVProductsInstalled float64
[1. 2. 3. 5. 0. 4. 6. 7.]
AVProductsEnabled float64
[1. 2. 0. 4. 3. 5.]
HasTpm int64
[1 0]
CountryIdentifier int64
[ 29  93  86  88  18  97  78 164 107  80 171 169 157 178 158  43 147  12
 203 205 199   9  68 201 141  35 159 160 129  44 207  15  89   3 108  55
 195 214 148 100  16 155 190  95  53 204  94   8 152 110 184 132 122  57
  39  51  59  70  52 121 149 142  41   2 179 180  27 103 167 101 220 119
 173  84 166

In [ ]:
print(df['EngineVersion_0'].unique())
print(df['EngineVersion_1'].unique())
print(df['EngineVersion_2'].unique())
print(df['EngineVersion_3'].unique())
print(df['AppVersion_0'].unique())
print(df['AppVersion_1'].unique())
print(df['AppVersion_2'].unique())
print(df['AppVersion_3'].unique())
print(df['AvSigVersion_0'].unique())
print(df['AvSigVersion_1'].unique())
print(df['AvSigVersion_2'].unique())
print(df['AvSigVersion_3'].unique())

In [ ]:
df['EngineVersion_0'] = df['EngineVersion'].apply(break_and_get_part, position=0)
df['EngineVersion_1'] = df['EngineVersion'].apply(break_and_get_part, position=1)
df['EngineVersion_2'] = df['EngineVersion'].apply(break_and_get_part, position=2)
df['EngineVersion_3'] = df['EngineVersion'].apply(break_and_get_part, position=3)

df['AppVersion_0'] = df['AppVersion'].apply(break_and_get_part, position=0)
df['AppVersion_1'] = df['AppVersion'].apply(break_and_get_part, position=1)
df['AppVersion_2'] = df['AppVersion'].apply(break_and_get_part, position=2)
df['AppVersion_3'] = df['AppVersion'].apply(break_and_get_part, position=3)

df['AvSigVersion_0'] = df['AvSigVersion'].apply(break_and_get_part, position=0)
df['AvSigVersion_1'] = df['AvSigVersion'].apply(break_and_get_part, position=1)
df['AvSigVersion_2'] = df['AvSigVersion'].apply(break_and_get_part, position=2)
df['AvSigVersion_3'] = df['AvSigVersion'].apply(break_and_get_part, position=3)

drop_list.append('EngineVersion')
drop_list.append('AppVersion')
drop_list.append('AvSigVersion')

In [ ]:
def break_and_get_part(value, position):
    return value.split(".")[position]

In [4]:
# Now analyze one column at a time
start_cnt = 0
show_cnt = 5
print(df.iloc[:5,start_cnt:start_cnt+show_cnt])
for y in df.columns[start_cnt:start_cnt+show_cnt]:
    print(y, df[y].dtype)
start_cnt += show_cnt

    ProductName  IsBeta  RtpStateBitfield  IsSxsPassiveMode  \
0  win8defender       0               7.0                 0   
1  win8defender       0               7.0                 0   
2  win8defender       0               7.0                 0   
3  win8defender       0               7.0                 0   
4  win8defender       0               7.0                 0   

   AVProductStatesIdentifier  
0                    53447.0  
1                    53447.0  
2                    53447.0  
3                    53447.0  
4                    53447.0  
ProductName object
IsBeta int64
RtpStateBitfield float64
IsSxsPassiveMode int64
AVProductStatesIdentifier float64


In [3]:
# Blindly replace all nan with 0.
# This has to be change later.
df = df.replace(np.nan, 0)

In [ ]:
# Remove columns with only 1 unique value
# Should do in the end also.
# TODO
# df.dropna(thresh=2)
for col in df:
    if(len(df.loc[:,col].unique()) == 1):
        print('Dropping: ' + col)
        drop_list.append(col)

In [ ]:
# Drop MachineIdentifier
drop_list.append('MachineIdentifier')

In [ ]:
drop_list = []

import operator 
# Find all the NaN
# See what percentage of each column is NaN
x_pct = {}
length = len(df)
for y in df.columns:
    if df[y].isnull().sum() > 0:
        x_cnt = df[y].isnull().sum()
        x_pct[y] = 100.0*x_cnt/length
        
x_pct_list = sorted(x_pct.items(), key=operator.itemgetter(1), reverse=True)
      
# Print the features, where more than 80% of the data is NaN    
for f, val in x_pct_list:
    if val > 80:
        print(f, val)
        drop_list.append(f)
        print('Dropping: ' + f)
        print('------')

In [ ]:
pd.set_option('max_colwidth',1000)

In [ ]:
import matplotlib.pyplot as plt
# Percentage of nulls
print(type(df.isnull().sum()/df.shape[0]))
plt.figure()
# (df.isnull().sum()/df.shape[0]).plot.hist()
print((df.isnull().sum()/df.shape[0]).sort_values())
(df.isnull().sum()/df.shape[0]).sort_values().plot()
plt.show()

In [ ]:
# Delete columns, where all the entries are 0.
df = df.loc[:, (df != 0).any(axis=0)]
print(df.shape)
df.info()

In [ ]:
file = 'data/train.csv'
df = pd.read_csv(file)

In [ ]:
import pandas as pd
import glob
import shutil
import sys
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import Imputer